In [1]:
from torchkge.data_structures import SmallKG, KnowledgeGraph
from torchkge.models.bilinear import ComplExModel
import pandas as pd

/home/rootdaenib/anaconda3/envs/test/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv('./kg.csv')

torchSmallKG = SmallKG(data['head'], data['tail'], data['type'])

In [3]:
tempData = data.rename(columns={'head':'from', 'tail':'to', 'type':'rel'})
print(tempData.columns)
torchKG = KnowledgeGraph(df=tempData)

Index(['from', 'rel', 'to'], dtype='object')


In [4]:
# complexModel = ComplExModel(emb_dim=100, n_entities=len(
    # set(torchKG.head_idx + torchSmallKG.tail_idx)), n_relations=len(set(torchSmallKG.relations)))
complexModelKG = ComplExModel(emb_dim=100, n_entities=torchKG.n_ent, n_relations=torchKG.n_rel)


complexModel.cuda()

In [5]:
complexModelKG.train(True)

ComplExModel(
  (re_ent_emb): Embedding(286, 100)
  (im_ent_emb): Embedding(286, 100)
  (re_rel_emb): Embedding(60, 100)
  (im_rel_emb): Embedding(60, 100)
)

In [6]:
from torchkge.utils import MarginLoss
from torch import cuda
criteria = MarginLoss(margin=1.0)

In [7]:
# if cuda.is_available():
#     cuda.empty_cache()
#     complexModelKG.cuda()
#     criteria.cuda()

In [8]:
import torch
from torch.optim import Adam
optimizer = Adam(complexModelKG.parameters(), lr=0.001)

In [9]:
from torchkge.sampling import BernoulliNegativeSampler
from torchkge.utils import DataLoader
from tqdm.autonotebook import tqdm
# torchKG = KnowledgeGraph(df=tempData).to('cuda')
sampler = BernoulliNegativeSampler(torchKG)
dataloader = DataLoader(torchKG, batch_size=10)
iterator = tqdm(range(1000), unit='epoch')
for epoch in iterator:
    runningLoss = 0.0
    for batch in dataloader:
        head, tail, relation = batch[0], batch[1], batch[2]
        # print(head.device, tail.device, relation.device, dataloader.use_cuda)
        numHead, numTail = sampler.corrupt_batch(head, tail, relation)
        optimizer.zero_grad()
        pos, neg = complexModelKG(head, tail, relation, numHead, numTail)
        loss = criteria(pos, neg)
        loss.backward()
        optimizer.step()
        runningLoss += loss.item()
    iterator.set_description('Epoch %d, loss %.5f' % (epoch, runningLoss/len(dataloader)))
    

Epoch 999, loss 0.06854: 100%|██████████| 1000/1000 [01:33<00:00, 10.73epoch/s]


In [17]:
from torchkge.models.deep import ConvKBModel

In [26]:
convModel = ConvKBModel(emb_dim=100, n_filters=4,n_entities=torchKG.n_ent, n_relations=torchKG.n_rel)

In [37]:
import torch
from torch.optim import Adam
optimizer = Adam(convModel.parameters(), lr=0.0001, amsgrad=True, weight_decay=0.000001)


In [38]:
from torchkge.sampling import BernoulliNegativeSampler
from torchkge.utils import DataLoader
from tqdm.autonotebook import tqdm
# torchKG = KnowledgeGraph(df=tempData).to('cuda')
sampler = BernoulliNegativeSampler(torchKG)
dataloader = DataLoader(torchKG, batch_size=20)
iterator = tqdm(range(100000), unit='epoch')
for epoch in iterator:
    runningLoss = 0.0
    for batch in dataloader:
        head, tail, relation = batch[0], batch[1], batch[2]
        # print(head.device, tail.device, relation.device, dataloader.use_cuda)
        numHead, numTail = sampler.corrupt_batch(head, tail, relation)
        optimizer.zero_grad()
        pos, neg = convModel(head, tail, relation, numHead, numTail)
        loss = criteria(pos, neg)
        loss.backward()
        optimizer.step()
        runningLoss += loss.item()
    iterator.set_description('Epoch %d, loss %.5f' %
                             (epoch, runningLoss/len(dataloader)))


Epoch 99999, loss 4.93107: 100%|██████████| 100000/100000 [1:34:29<00:00, 17.64epoch/s]
